# Synth Machine Quickstart

In [1]:
from synth_machine.machine import Synth
import logging
import json

/home/simonbutt/.cache/pypoetry/virtualenvs/tofuagent-SeWjU9e5-py3.12/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
# Set logging to debug to view all internals in transition trigger.
logging.basicConfig(level=logging.ERROR)

## Synth machine config
#### Entire synth config spec can be found: https://hiresynth.ai/docs/spec.html

In [3]:
config = {
    "default_model_config": {
        "max_tokens": 50
    },
    "initial_state": "start",
    "states": [
        {
            "name": "start"
        },
        {
            "name": "end"
        }
    ],
    "transitions": [
        {
            "source": "start",
            "dest": "end",
            "trigger": "run",
            "inputs": [
                {
                    "key": "input1"
                }
            ],
            "outputs": [
                {
                    "key": "output1",
                    "prompt": "Tell me about {{ input1 }}",
                    "schema": {
                        "type": "string"
                    }
                }
            ]
        },
        {
            "source": "end",
            "dest": "end",
            "trigger": "mock_run",
            "inputs": [
                {
                    "key": "output1"
                }
            ],
            "outputs": [
                {
                    "key": "output2",
                    "prompt": "Walk me through: {{ output1 }}",
                    "schema": {
                        "type": "string"
                    },
                    "model_config": {
                        "executor": "mock"
                    }
                }
            ]
        }
    ]
}

## Tools
#### Tools are RestAPIs and can be added by providing the JSON API schema

In [4]:
tool_spec = json.load(open("./tools/tofuTool/openapi_schema.json"))

In [5]:
tool_spec

{'openapi': '3.1.0',
 'info': {'title': 'FastAPI', 'version': '0.1.0'},
 'paths': {'/tofu/validate': {'get': {'summary': ' Validate',
    'operationId': '_validate_tofu_validate_get',
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'schema': {'type': 'object',
         'title': 'Response  Validate Tofu Validate Get'}}}}}}},
  '/tofu/plan': {'get': {'summary': ' Plan',
    'description': 'Endpoint to generate and show Terraform plan in JSON format.',
    'operationId': '_plan_tofu_plan_get',
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'schema': {'type': 'object',
         'title': 'Response  Plan Tofu Plan Get'}}}}}}}}}

### Defining the Agent

In [6]:
agent = Synth(
    config,
    tools=[tool_spec]
)

### Agent state and possible triggers

In [7]:
print(f"Current State: {agent.current_state()}")
print(f"Available triggers: {agent.interfaces_for_available_triggers()}")

Current State: start
Available triggers: [{'source': 'start', 'dest': 'end', 'trigger': 'run', 'inputs': [{'key': 'input1'}], 'outputs': [{'key': 'output1', 'prompt': 'Tell me about {{ input1 }}', 'schema': {'type': 'string'}}]}]


### Memory

In [8]:
agent.memory

{}

### Agent Trigger

In [9]:
await agent.trigger("run", {"input1": "FOX news"})

{'output1': ' Fox News is an American cable and satellite news television channel owned by the Fox Entertainment Group, a subsidiary of 21st Century Fox. The channel broadcasts primarily from studios at 1211 Avenue of the Americas'}

In [10]:
print(f"Current State: {agent.current_state()}")
agent.memory

Current State: end


{'input1': 'FOX news',
 'output1': ' Fox News is an American cable and satellite news television channel owned by the Fox Entertainment Group, a subsidiary of 21st Century Fox. The channel broadcasts primarily from studios at 1211 Avenue of the Americas'}

### Streaming Trigger
#### The streaming trigger provides all the events coming during the changing of state.
This includes:
- **MACHINE_UPDATES**: Includes all changes to memory
- **INPUTS**: The input for that transition
- **SAFETY**: (if set) output of the moderation on either the prompt or output
- **CHUNK**: Each LLM token will stream as a CHUNK

In [11]:
async for value in agent.streaming_trigger(
    "mock_run", {}
):
    print(value)

['MACHINE_UPDATE', [{'source': 'end', 'dest': 'end', 'trigger': 'mock_run', 'inputs': [{'key': 'output1'}], 'outputs': [{'key': 'output2', 'prompt': 'Walk me through: {{ output1 }}', 'schema': {'type': 'string'}, 'model_config': {'executor': 'mock'}}]}], {'input1': 'FOX news', 'output1': ' Fox News is an American cable and satellite news television channel owned by the Fox Entertainment Group, a subsidiary of 21st Century Fox. The channel broadcasts primarily from studios at 1211 Avenue of the Americas'}, 'end', 'mock_run']
['INPUTS', {'output1': ' Fox News is an American cable and satellite news television channel owned by the Fox Entertainment Group, a subsidiary of 21st Century Fox. The channel broadcasts primarily from studios at 1211 Avenue of the Americas'}]
[<YieldTasks.SET_ACTIVE_OUTPUT: 'SET_ACTIVE_OUTPUT'>, 'output2']
[<YieldTasks.SET_MEMORY: 'SET_MEMORY'>, 'output2', {}]
['SAFETY', 'SUCCESS', 'PROMPT', '{"hate": {"score": "negligible", "flagged": false}, "harassment": {"scor

### That's the basics of HireSynth `synth_machine`!